In [11]:
import numpy as np
import tensorflow as tf
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import pickle
import os
from tqdm import tqdm
import time
from scipy.stats import gaussian_kde

In [12]:
model = tf.keras.models.load_model('D:/ExponentiatedNonPeriodicNoisy/5-conv-512-channels-5-dense-100-epochs-1555584997.model')

In [13]:
pickle_in = open('D:/ExponentiatedNonPeriodicNoisy/Testing/X_test.pickle','rb')
X_test = pickle.load(pickle_in)

pickle_in = open('D:/ExponentiatedNonPeriodicNoisy/Testing/y_test.pickle','rb')
y_test = pickle.load(pickle_in)

print(y_test)
#y_test = y_test[0]

[[0.10128430082404583, 0.11309985236964215, 0.6677964407198851, 0.7071248036844345, 0.7920459948279429, 0.7504453337951883, 0.4185351048894268, 0.5287633038973606, 0.19363631308201845, 0.3673627427962324, 0.004073034764868644, 0.10568659416201742, 0.7439979381753844, 0.9277689673108689, 0.9758791640676543, 0.18101163765263528, 0.6995002023109729, 0.3464385678704772, 0.8874471198527291, 0.4195769957456513, 0.27174103691161533, 0.556845410350272, 0.6298502088124087, 0.3409344157456484, 0.6772588826562507, 0.5043553201372266, 0.43509983148987885, 0.6950202434805038, 0.1305527336778689, 0.9820823747866008, 0.5421405063442561, 0.3501163172040459, 0.31416745548529945, 0.7473435073049537, 0.13351316717581452, 0.7574553027826665, 0.9781356946997977, 0.1883178405082344, 0.06968901649324755, 0.1630070561236875, 0.1513291021142108, 0.6872804392420641, 0.37292964685649876, 0.3262276112662078, 0.15984697813344018, 0.8614507128423463, 0.8816658016196615, 0.3520867200663127, 0.18454587059411542, 0.27

In [ ]:
evaluations = model.evaluate(X_test, y_test)

In [ ]:
print(evaluations)

np.savetxt('D:/ExponentiatedNonPeriodicNoisy/evaluations.txt',evaluations)

In [ ]:
predictions = model.predict(X_test)

In [10]:
print(predictions)

np.savetxt('D:/ExponentiatedNonPeriodicNoisy/predictions.txt',np.reshape(np.asarray(predictions),(2,-1)).T)

[array([[0.1477148 ],
       [0.03356427],
       [0.6861657 ],
       ...,
       [0.4236743 ],
       [0.66379374],
       [0.6829    ]], dtype=float32), array([[1.4032738],
       [2.579177 ],
       [2.0465841],
       ...,
       [1.1726512],
       [1.0834452],
       [1.3716688]], dtype=float32)]


In [14]:
predictions = np.loadtxt('D:/ExponentiatedNonPeriodicNoisy/predictions.txt')
evaluations = np.loadtxt('D:/ExponentiatedNonPeriodicNoisy/evaluations.txt')

In [15]:
total_loss = evaluations[0]
H_loss = evaluations[1]
sigma_loss = evaluations[2]

In [16]:
H_predicted = predictions[:,0]
sigma_predicted = predictions[:,1]
print(H_predicted)

[0.14771479 0.03356427 0.68616569 ... 0.42367429 0.66379374 0.68290001]


In [17]:
print(H_predicted.flatten())

[0.14771479 0.03356427 0.68616569 ... 0.42367429 0.66379374 0.68290001]


In [18]:
y_test

[[0.10128430082404583,
  0.11309985236964215,
  0.6677964407198851,
  0.7071248036844345,
  0.7920459948279429,
  0.7504453337951883,
  0.4185351048894268,
  0.5287633038973606,
  0.19363631308201845,
  0.3673627427962324,
  0.004073034764868644,
  0.10568659416201742,
  0.7439979381753844,
  0.9277689673108689,
  0.9758791640676543,
  0.18101163765263528,
  0.6995002023109729,
  0.3464385678704772,
  0.8874471198527291,
  0.4195769957456513,
  0.27174103691161533,
  0.556845410350272,
  0.6298502088124087,
  0.3409344157456484,
  0.6772588826562507,
  0.5043553201372266,
  0.43509983148987885,
  0.6950202434805038,
  0.1305527336778689,
  0.9820823747866008,
  0.5421405063442561,
  0.3501163172040459,
  0.31416745548529945,
  0.7473435073049537,
  0.13351316717581452,
  0.7574553027826665,
  0.9781356946997977,
  0.1883178405082344,
  0.06968901649324755,
  0.1630070561236875,
  0.1513291021142108,
  0.6872804392420641,
  0.37292964685649876,
  0.3262276112662078,
  0.1598469781334401

In [19]:
from scipy.interpolate import griddata

def grid(x, y, z, resX=100, resY=100):
    "Convert 3 column data to matplotlib grid"
    xi = np.linspace(min(x), max(x), resX)
    yi = np.linspace(min(y), max(y), resY)
    Z = griddata((x, y), z, (xi[None,:], yi[:,None]),method="linear")
    X, Y = np.meshgrid(xi, yi)
    return X, Y, Z

In [23]:
xy = np.vstack([y_test[0],H_predicted.flatten()])
z = gaussian_kde(xy)(xy)

X, Y, Z = grid(y_test[0], H_predicted.flatten(), z)

plt.figure(dpi = 250)
plt.contourf(X, Y, Z,alpha=0)
plt.contour(X,Y,Z, colors='k',linewidths=0.7,linestyles='dashed')
plt.scatter(y_test[0],H_predicted.flatten(),c=z/np.size(z),s=5)
plt.colorbar()
plt.text(0.021, 0.95, 'RMSE = '+ '{:.3g}'.format(np.sqrt(H_loss)),bbox = dict(facecolor='white'))
plt.plot([0,1],[0,1],c='k')
plt.ylim([0,1])
plt.xlim([0,1])
plt.xlabel(r'$H_{true}$')
plt.ylabel(r'$H_{derived}$')
plt.tight_layout()
plt.grid()
plt.savefig('D:/ExponentiatedNonPeriodicNoisy/H-Performance.png')
plt.close()

In [24]:
xy = np.vstack([y_test[1],sigma_predicted.flatten()])
z = gaussian_kde(xy)(xy)

X, Y, Z = grid(y_test[1], sigma_predicted.flatten(), z)

plt.figure(dpi = 250)
plt.contourf(X, Y, Z,alpha=0)
plt.contour(X,Y,Z, colors='k',linewidths=0.7,linestyles='dashed')
plt.scatter(y_test[1],sigma_predicted.flatten(),c=z/np.size(z),s=5)
plt.colorbar()
plt.text(0.021*3, 0.95*3, 'RMSE = '+ '{:.3g}'.format(np.sqrt(sigma_loss/3)),bbox = dict(facecolor='white'))
plt.plot([0,3],[0,3],c='k')
plt.ylim([0,3])
plt.xlim([0,3])
plt.xlabel(r'$\sigma_{true}$')
plt.ylabel(r'$\sigma_{derived}$')
plt.tight_layout()
plt.grid()
plt.savefig('D:/ExponentiatedNonPeriodicNoisy/Sigma-Performance.png')
plt.close()

In [25]:
y_test_1 = y_test[0]
y_test_2 = y_test[1]

In [26]:
histogram_true_1 = np.histogram(y_test_1, bins = 5)
histogram_true_2 = np.histogram(y_test_2, bins = 5)

In [27]:
print(histogram_true_1)

(array([395, 365, 423, 411, 406], dtype=int64), array([4.91572944e-04, 2.00187299e-01, 3.99883025e-01, 5.99578751e-01,
       7.99274478e-01, 9.98970204e-01]))


In [28]:
bins_true_1 = histogram_true_1[1]
bins_true_2 = histogram_true_2[1]
print(bins_true_1)

[4.91572944e-04 2.00187299e-01 3.99883025e-01 5.99578751e-01
 7.99274478e-01 9.98970204e-01]


In [29]:
ind_1 = np.digitize(y_test_1,bins_true_1)
ind_2 = np.digitize(y_test_2,bins_true_2)
print(ind_1)

[1 1 4 ... 2 4 4]


In [30]:
ind_1

array([1, 1, 4, ..., 2, 4, 4], dtype=int64)

In [31]:
first_indices_1 = np.where(ind_1==1)[0]
second_indices_1 = np.where(ind_1==2)[0]
third_indices_1 = np.where(ind_1==3)[0]
fourth_indices_1 = np.where(ind_1==4)[0]
fifth_indices_1 = np.where(ind_1==5)[0]

In [32]:
first_indices_2 = np.where(ind_2==1)[0]
second_indices_2 = np.where(ind_2==2)[0]
third_indices_2 = np.where(ind_2==3)[0]
fourth_indices_2 = np.where(ind_2==4)[0]
fifth_indices_2 = np.where(ind_2==5)[0]

In [33]:
first_indices_1

array([   0,    1,    8,   10,   11,   15,   28,   34,   37,   38,   39,
         40,   44,   48,   59,   63,   75,   78,   79,   85,   87,   91,
         95,  100,  105,  106,  110,  116,  117,  130,  132,  142,  143,
        144,  150,  153,  162,  166,  169,  171,  172,  173,  179,  184,
        185,  193,  207,  213,  216,  222,  228,  232,  237,  247,  249,
        254,  266,  268,  271,  277,  279,  283,  293,  301,  308,  310,
        312,  321,  346,  347,  349,  363,  364,  365,  368,  369,  372,
        379,  383,  384,  385,  391,  392,  398,  399,  402,  410,  417,
        426,  442,  443,  445,  448,  453,  465,  466,  473,  485,  506,
        517,  521,  526,  530,  532,  536,  545,  546,  556,  558,  568,
        569,  570,  579,  582,  590,  597,  598,  599,  600,  612,  616,
        620,  623,  628,  633,  637,  647,  653,  655,  660,  661,  663,
        664,  665,  668,  669,  670,  673,  687,  694,  701,  709,  711,
        712,  713,  714,  725,  741,  744,  745,  7

In [34]:
first_bin_true_1 = np.asarray(y_test_1)[first_indices_1]
second_bin_true_1 = np.asarray(y_test_1)[second_indices_1]
third_bin_true_1 = np.asarray(y_test_1)[third_indices_1]
fourth_bin_true_1 = np.asarray(y_test_1)[fourth_indices_1]
fifth_bin_true_1 = np.asarray(y_test_1)[fifth_indices_1]

In [35]:
first_bin_true_2 = np.asarray(y_test_2)[first_indices_2]
second_bin_true_2 = np.asarray(y_test_2)[second_indices_2]
third_bin_true_2 = np.asarray(y_test_2)[third_indices_2]
fourth_bin_true_2 = np.asarray(y_test_2)[fourth_indices_2]
fifth_bin_true_2 = np.asarray(y_test_2)[fifth_indices_2]

In [36]:
first_bin_pred_1 = np.asarray(H_predicted.flatten())[first_indices_1]
second_bin_pred_1 = np.asarray(H_predicted.flatten())[second_indices_1]
third_bin_pred_1 = np.asarray(H_predicted.flatten())[third_indices_1]
fourth_bin_pred_1 = np.asarray(H_predicted.flatten())[fourth_indices_1]
fifth_bin_pred_1 = np.asarray(H_predicted.flatten())[fifth_indices_1]

In [37]:
first_bin_pred_2 = np.asarray(sigma_predicted.flatten())[first_indices_2]
second_bin_pred_2 = np.asarray(sigma_predicted.flatten())[second_indices_2]
third_bin_pred_2 = np.asarray(sigma_predicted.flatten())[third_indices_2]
fourth_bin_pred_2 = np.asarray(sigma_predicted.flatten())[fourth_indices_2]
fifth_bin_pred_2 = np.asarray(sigma_predicted.flatten())[fifth_indices_2]

In [38]:
first_bin_pred_1

array([ 0.14771479,  0.03356427,  0.11318073, -0.02225512,  0.09130779,
        0.20613718,  0.07373807,  0.17186072,  0.27389863,  0.09990546,
        0.00660756,  0.17487714,  0.07123494,  0.0646404 ,  0.20562544,
        0.00638926,  0.09351382,  0.23659892,  0.1485064 ,  0.18496123,
        0.28797257,  0.04300883,  0.064913  ,  0.07607111,  0.10683715,
        0.24894641, -0.01115769, -0.09167874,  0.14429602,  0.22750136,
        0.27904248, -0.01672959,  0.14029163,  0.1465537 ,  0.09734976,
        0.18618962,  0.03393945,  0.04165015,  0.23140481,  0.14982995,
        0.13346806,  0.11804891,  0.05796772,  0.14445257,  0.09403411,
        0.20756593,  0.13827908,  0.0876424 , -0.02161127,  0.16766122,
        0.06951106,  0.16023695,  0.1218774 ,  0.05238253,  0.04276308,
        0.14866155,  0.10860744,  0.15416482,  0.30113775, -0.00973156,
        0.17496598,  0.12489444,  0.13406536,  0.15134463, -0.00305295,
        0.18510938,  0.21448877,  0.26338103,  0.02559203,  0.09

In [39]:
first_bin_diff_1 = first_bin_pred_1-first_bin_true_1
second_bin_diff_1 = second_bin_pred_1-second_bin_true_1
third_bin_diff_1 = third_bin_pred_1-third_bin_true_1
fourth_bin_diff_1 = fourth_bin_pred_1-fourth_bin_true_1
fifth_bin_diff_1 = fifth_bin_pred_1-fifth_bin_true_1

In [40]:
first_bin_diff_2 = first_bin_pred_2-first_bin_true_2
second_bin_diff_2 = second_bin_pred_2-second_bin_true_2
third_bin_diff_2 = third_bin_pred_2-third_bin_true_2
fourth_bin_diff_2 = fourth_bin_pred_2-fourth_bin_true_2
fifth_bin_diff_2 = fifth_bin_pred_2-fifth_bin_true_2

In [41]:
data_to_plot_1 = [first_bin_diff_1,second_bin_diff_1,third_bin_diff_1,fourth_bin_diff_1,fifth_bin_diff_1]

In [42]:
data_to_plot_2 = [first_bin_diff_2,second_bin_diff_2,third_bin_diff_2,fourth_bin_diff_2,fifth_bin_diff_2]

In [43]:
fig = plt.figure(1, dpi=200)
ax = fig.add_subplot(111)
bp=ax.boxplot(data_to_plot_1)
ax.axhline()
ax.set_xticklabels(['{:.2f}'.format((bins_true_1[1]+bins_true_1[0])/2),'{:.2f}'.format((bins_true_1[2]+bins_true_1[1])/2),'{:.2f}'.format((bins_true_1[3]+bins_true_1[2])/2),'{:.2f}'.format((bins_true_1[4]+bins_true_1[3])/2),'{:.2f}'.format((bins_true_1[5]+bins_true_1[4])/2)])
ax.set_xlabel(r'$H_{true}$')
ax.set_ylabel(r'$H_{derived}-H_{true}$')
#ax.set_ylim(-0.4,0.4)
#fig.savefig('C:/Users/C1764397/Workshop/PhD/Data/fBmFields/2DExponentiated/Periodic/Box-plot-H-Exponentiated-{}'.format(NAME,), bbox_inches='tight')
fig.show()

C:\Users\C1764397\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\figure.py:418: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "


In [44]:
fig = plt.figure(1, dpi=200)
ax = fig.add_subplot(111)
bp=ax.boxplot(data_to_plot_2)
ax.axhline()
ax.set_xticklabels(['{:.2f}'.format((bins_true_2[1]+bins_true_2[0])/2),'{:.2f}'.format((bins_true_2[2]+bins_true_2[1])/2),'{:.2f}'.format((bins_true_2[3]+bins_true_2[2])/2),'{:.2f}'.format((bins_true_2[4]+bins_true_2[3])/2),'{:.2f}'.format((bins_true_2[5]+bins_true_2[4])/2)])
ax.set_xlabel(r'$\sigma_{true}$')
ax.set_ylabel(r'$\sigma_{derived}-\sigma_{true}$')
#ax.set_ylim(-3,3)
#fig.savefig('C:/Users/C1764397/Workshop/PhD/Data/fBmFields/2DExponentiated/Periodic/Box-plot-sigma-Exponentiated-{}'.format(NAME,), bbox_inches='tight')
fig.show()

C:\Users\C1764397\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
C:\Users\C1764397\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\figure.py:418: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "


In [45]:
plt.figure(dpi = 200)
plt.scatter(sigma_predicted.flatten(),y_test[1]-sigma_predicted.flatten())
plt.show()